In [35]:
from datasets import load_dataset
import dspy
import openai
import os
import re
import pandas as pd
import json
import random
from dotenv import load_dotenv
import glob

In [ ]:
load_dotenv('/Users/hungcoreft/workspace/ood-modification/eval/.env')

In [37]:

openai.api_key = os.getenv('OPENAI_API_KEY')
openai.organization = os.getenv('OPENAI_ORGANIZATION')


In [28]:
lm = dspy.LM('openai/gpt-4o', temperature=0, max_tokens=1023)
dspy.configure(lm=lm)

In [7]:
ds = pd.read_json('../preprocessing/train_dev_test_data/dialogue/test.json')
ds = ds.to_dict('records')


In [ ]:
samples_with_contradiction = []
samples_no_contradiction = []
for i, x in enumerate(ds):
    if x["is_contradiction"]:
        samples_with_contradiction.append((i, x))
    else:
        samples_no_contradiction.append((i, x))
print(len(samples_with_contradiction), len(samples_no_contradiction))

In [12]:
samples = samples_with_contradiction + samples_no_contradiction
random.shuffle(samples)

In [13]:
label_map = {'is_contradiction': 1, 'no_contradiction': 0}

In [8]:
def remove_space(text):
    """Clean up spacing and formatting in dialogue text."""
    lines = text.split('\n')
    cleaned_lines = []
    
    for line in lines:
        # Remove multiple spaces
        cleaned = ' '.join(line.split())
        
        # Fix spacing around punctuation
        cleaned = re.sub(r'\s+([.,!?])', r'\1', cleaned)
        cleaned = re.sub(r'([.,!?])\s+', r'\1 ', cleaned)
        
        # Fix contractions
        cleaned = re.sub(r'\s*\'\s*s\b', "'s", cleaned)
        cleaned = re.sub(r'\s*n\s*\'\s*t\b', "n't", cleaned)
        cleaned = re.sub(r'\s*\'\s*ve\b', "'ve", cleaned)
        cleaned = re.sub(r'\s*\'\s*re\b', "'re", cleaned)
        cleaned = re.sub(r'\s*\'\s*ll\b', "'ll", cleaned)
        cleaned = re.sub(r'\s*\'\s*d\b', "'d", cleaned)
        cleaned = re.sub(r'\s*\'\s*m\b', "'m", cleaned)
        
        # Fix spaces around parentheses
        cleaned = re.sub(r'\(\s+', '(', cleaned)
        cleaned = re.sub(r'\s+\)', ')', cleaned)
        
        # Remove leading/trailing whitespace
        cleaned = cleaned.strip()
        
        cleaned_lines.append(cleaned)
        
    return '\n'.join(cleaned_lines)


In [15]:
examples = [
    dspy.Example({ 
                  "dialogue" : remove_space('\n'.join(r["dialogue"])), 
                  "label": label_map[r['label']]
                }
                  ).with_inputs("dialogue") 
    for i,r in samples
    ]

In [ ]:
example = examples[59]
for k, v in example.items():
    print(f"\n{k.upper()}:\n")
    print(v)


In [9]:
def extract_prediction(text):
    matches = re.findall(r'\b[0-2]\b', text)
    # print(matches)
    parsed_answer = matches[-1] if matches else ""
    return parsed_answer

In [10]:
def eval_metric(true, prediction, trace=None):
    pred = prediction.label
    # Find all occurrences of 0 or 1 that are bounded by word boundaries (\b)
    # \b ensures we only match standalone 0s and 1s, not numbers like 10 or 01
    # [0-1] matches either 0 or 1
    matches = re.findall(r'\b[0-2]\b', pred)
    # print(matches)
    parsed_answer = matches[-1] if matches else ""
    
    # print(parsed_answer)
    return parsed_answer == str(true.label)

# Evaluate the original test set

In [11]:
from dspy.evaluate import Evaluate

In [20]:
class Dialogue(dspy.Signature):
    """Does the last utterance contradict the dialogue context? Answer with 1 if contradict, 0 if not contradict"""
    dialogue = dspy.InputField()
    label = dspy.OutputField(prefix = 'Answer:')

In [291]:
class SimpleDialogue(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict(Dialogue)

    def forward(self, dialogue):

        return self.prog(dialogue = dialogue)


In [292]:
simple_dialogue = SimpleDialogue()

In [ ]:
pred = simple_dialogue(dialogue = example.dialogue)
print("\nDIALOGUE:\n")
print(example.dialogue)

print("\nANSWER:\n")
print(example.label)

print("\nPREDICTION:\n")
print(pred)


In [ ]:
eval_metric(example, pred)

In [ ]:
evaluate = Evaluate(devset= examples, metric=eval_metric, num_threads=6, display_progress=True, display_table=10, return_outputs= True, return_all_scores=True)
results = evaluate(simple_dialogue)


In [ ]:
items = []
for sample in results[1]:
    item = {}
    print(sample[0])
    dialog = sample[0]['dialogue']
    label = sample[0]['label']
    pred = sample[1]['label']
    item['dialog'] = dialog
    item['label'] = label
    item['pred'] = pred
    items.append(item)
df_result = pd.DataFrame(data = items)
df_result.to_csv('results/dialogue/gpt4o-0shot-dialogue.csv')

In [293]:
class CoTDialogue(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(Dialogue)

    def forward(self, dialogue):

        return self.prog(dialogue = dialogue)


In [ ]:
cot_dialogue = CoTDialogue()
pred = cot_dialogue(dialogue = example.dialogue)
print("\nDIALOGUE:\n")
print(example.dialogue)
print("\nANSWER:\n")
print(example.label)
print("\nPREDICTION:\n")
print(pred)


In [ ]:
evaluate = Evaluate(devset= examples, metric=eval_metric, num_threads=6, display_progress=True, display_table=10, return_outputs= True, return_all_scores=True)
results = evaluate(cot_dialogue)


In [ ]:
items = []
for sample in results[1]:
    item = {}
    print(sample[0])
    dialog = sample[0]['dialogue']
    label = sample[0]['label']
    pred = sample[1]['label']
    rationale = sample[1]['reasoning']
    item['dialog'] = dialog
    item['label'] = label
    item['pred'] = pred
    item['reasoning'] = rationale
    items.append(item)
df_result = pd.DataFrame(data = items)
df_result.to_csv('results/dialogue/gpt4o-0shot-cot-dialogue.csv')

In [12]:
def append_person(ds):
    for i,sample in enumerate(ds):
        # print(sample)
        modified_dialog = sample['dialog_context'] + [sample['modified_text']]
        original_dialog = sample['dialog_context'] + [sample['original_text']]
        for j, turn in enumerate(modified_dialog):
            turn = 'agent ' + str(j%2) + ': ' + turn
            modified_dialog[j] = turn
        for j, turn in enumerate(original_dialog):
            turn = 'agent ' + str(j%2) + ': ' + turn
            original_dialog[j] = turn
        ds[i]['original_dialog'] = '\n'.join(original_dialog)
        ds[i]['original_dialog'] = remove_space(ds[i]['original_dialog'])
        ds[i]['modified_dialog'] = '\n'.join(modified_dialog)
        ds[i]['modified_dialog'] = remove_space(ds[i]['modified_dialog'])

    return ds


# Evaluate by modification

## Without label change

In [29]:
def evaluate_modified_set(ds, program):
    examples = [
    dspy.Example({ 
                  "dialogue" : r['modified_dialog'], 
                  "original_dialogue": r['original_dialog'],
                  "label": int(r['label']),
                  "modified_label": int(r['label'])
                }
                  ).with_inputs("dialogue") 
    for r in ds
    ]
    evaluate = Evaluate(devset= examples, metric=eval_metric, num_threads=6, display_progress=True, display_table=1, return_outputs= True, return_all_scores=True)
    results = evaluate(program)
    return results

In [30]:
class Dialogue(dspy.Signature):
    """Does the last utterance contradict the dialogue context? Answer with 1 if contradict, 0 if not contradict"""
    dialogue = dspy.InputField()
    label = dspy.OutputField(prefix = 'Answer:')

In [31]:
class SimpleDialogue(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict(Dialogue)

    def forward(self, dialogue):

        return self.prog(dialogue = dialogue)
simple_dialogue = SimpleDialogue()

In [ ]:
# Configure GPT-4 as the language model
lm = dspy.LM('openai/gpt-4o', temperature=0, max_tokens=300)
dspy.configure(lm=lm)

# Get all json files in the specified directory
json_files = glob.glob('../data/modified_data/dialogue/*_100.json')
original_pred_ds = pd.read_csv('results/dialogue/gpt4o-0shot-dialogue.csv', index_col=False)
original_pred_ds['dialog'] = original_pred_ds['dialog'].apply(remove_space)  # Replace 'your_function' with the actual function
# print(original_pred_ds['dialog'][1958])
for json_file in json_files:
    if any(x in json_file for x in ['grammatical_role', 'negation']):
        continue
 
    # Load the json file
    print(json_file)
    # with open(json_file, 'r') as f:
    #     data = json.load(f)
    if not any(x in json_file for x in ['capitalization', 'typo_bias', 'punctuation', 'grammatical_role', 'negation']):
        data = pd.read_json(json_file)[1]
        data = list(data)
    else:
        with open(json_file,'r') as f:
            data = json.load(f)
        # data = pd.read_json(json_file)
        # data = data.to_json(orient = 'records')
        # data = ast.literal_eval(data)
    # print(data)
    data = append_person(data)
    results_modified = evaluate_modified_set(data, simple_dialogue)
    items = []
    for sample in results_modified[1]:
        item = {}
        # print(sample[0])
        modified_dialog = sample[0]['dialogue']
        original_dialog = sample[0]['original_dialogue']

        label = sample[0]['label']
        pred = sample[1]['label']
        # rationale = sample[1]['reasoning']
        # original_pred = compare_dialog(original_pred_ds, original_pred_ds['dialog'], original_dialog)
        original_dialog = remove_space(original_dialog)
        # print(original_dialog)
        # print()
        original_pred = original_pred_ds.loc[original_pred_ds['dialog'] == original_dialog]['pred'].values[0]
        item['original_dialog'] = original_dialog
        item['modified_dialog'] = modified_dialog
        modified_pred = extract_prediction(pred)
        item['modified_label'] = label
        item['original_label'] = label
        item['modified_pred'] = modified_pred
        item['original_pred'] = original_pred
        # item['reasoning'] = sample[1]['reasoning']
        # item['reasoning'] = rationale
        items.append(item)
    
    df_result = pd.DataFrame(data=items)
    
    # Save results with filename based on input json
    output_filename = f"results/dialogue/gpt4o-0shot-{json_file.split('/')[-1].replace('.json', '')}.csv"
    df_result.to_csv(output_filename)


## With label change

In [38]:
def evaluate_modified_set(ds, program):
    examples = [
    dspy.Example({ 
                  "dialogue" : r['modified_dialog'], 
                  "original_dialogue": r['original_dialog'],
                  "label": int(r['modified_label']) if r.get('modified_label')!= None else int(r['label']),
                  "original_label": int(r['label']),
                  "index": r['index'],
                  "type": r['type'] if r.get('type') != None else None
                }
                  ).with_inputs("dialogue") 
    for r in ds
    ]
    evaluate = Evaluate(devset= examples, metric=eval_metric, num_threads=6, display_progress=True, display_table=1, return_outputs= True, return_all_scores=True)
    results = evaluate(program)
    return results

In [39]:
class Dialogue(dspy.Signature):
    """Does the last utterance contradict the dialogue context? Answer with 1 if contradict, 0 if not contradict"""
    dialogue = dspy.InputField()
    label = dspy.OutputField(prefix = 'Answer:')

In [40]:
class SimpleDialogue(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict(Dialogue)

    def forward(self, dialogue):

        return self.prog(dialogue = dialogue)
simple_dialogue = SimpleDialogue()

In [ ]:

# Configure GPT-4 as the language model
lm = dspy.LM('openai/gpt-4o', temperature=0, max_tokens=250)
dspy.configure(lm=lm)

# Get all json files in the specified directory
json_files = glob.glob('../data/modified_data/dialogue/*_100.json')
original_pred_ds = pd.read_csv('results/dialogue/gpt4o-0shot-dialogue.csv', index_col=False)
original_pred_ds['dialog'] = original_pred_ds['dialog'].apply(remove_space)  # Replace 'your_function' with the actual function
# print(original_pred_ds['dialog'][1958])
for json_file in json_files:
    if not any(x in json_file for x in ['geographical_bias']):
        continue
 
    # Load the json file
    print(json_file)
    # with open(json_file, 'r') as f:
    #     data = json.load(f)
    with open(json_file,'r') as f:
        raw_data = json.load(f)
    data = [dict(sample, index=idx) for idx, sample in raw_data]

        # data = pd.read_json(json_file)
        # data = data.to_json(orient = 'records')
        # data = ast.literal_eval(data)
    # print(data)
    data = append_person(data)
    results_modified = evaluate_modified_set(data, simple_dialogue)
    items = []
    for sample in results_modified[1]:
        item = {}
        # print(sample[0])
        modified_dialog = sample[0]['dialogue']
        original_dialog = sample[0]['original_dialogue']

        pred = sample[1]['label']
        # rationale = sample[1]['reasoning']
        # original_pred = compare_dialog(original_pred_ds, original_pred_ds['dialog'], original_dialog)
        original_dialog = remove_space(original_dialog)
        # print(original_dialog)
        # print()
        original_pred = original_pred_ds.loc[original_pred_ds['dialog'] == original_dialog]['pred'].values[0]
        item['original_dialog'] = original_dialog
        item['modified_dialog'] = modified_dialog
        modified_pred = extract_prediction(pred)
        item['modified_label'] = sample[0]['label']
        item['original_label'] = sample[0]['original_label']
        item['modified_pred'] = modified_pred
        item['original_pred'] = original_pred
        item['type'] = sample[0]['type']
        # item['reasoning'] = sample[1]['reasoning']
        # item['reasoning'] = rationale
        items.append(item)
    
    df_result = pd.DataFrame(data=items)
    
    # Save results with filename based on input json
    output_filename = f"results/dialogue/gpt4o-0shot-{json_file.split('/')[-1].replace('.json', '')}.csv"
    df_result.to_csv(output_filename)


# Aggregate results

In [54]:
from scipy import stats

In [ ]:
result_files = glob.glob('results/dialogue/gpt4o-0shot-*_100.csv')

aggregated_results = []

for file in result_files:
    # Extract modification type from filename
    mod_type = file.split('-')[-1].replace('.csv','')
    
    # Read results file
    df = pd.read_csv(file)

    # Calculate accuracies
    original_correct = (df['original_pred'] == df['original_label']).sum()
    modified_correct = (df['modified_pred'] == df['modified_label']).sum()
    total = len(df)

    original_acc = original_correct / total
    modified_acc = modified_correct / total
    
    # Calculate the difference between original_res and modified_res
    difference = -round(original_acc - modified_acc, 2)
    
    # Calculate percentage difference with respect to total samples
    pct_difference = -round((original_acc - modified_acc) / original_acc * 100, 2)
    
    # Perform t-test between original and modified predictions
    t_stat, p_value = stats.ttest_ind(
        (df['original_pred'] == df['original_label']).astype(float),
        (df['modified_pred'] == df['modified_label']).astype(float)
    )
    
    aggregated_results.append({
        'task': 'dialogue_contradiction_detection',
        'modification': mod_type,
        'original_res': round(original_acc, 2),
        'modified_res': round(modified_acc, 2),
        'difference': difference,  # Difference in accuracy
        'pct_difference': pct_difference,  # Percentage difference relative to total samples
        'p_value': p_value  # Add p-value from t-test
    })

# Create final results dataframe
results_df = pd.DataFrame(aggregated_results)

# Sort the results based on modification_name
modification_name = ['temporal_bias_100', 'geographical_bias_100','length_bias_100', 'typo_bias_100', 'capitalization_100', 'punctuation_100', 'derivation_100', 'compound_word_100','active_to_passive_100','grammatical_role_100', 'coordinating_conjunction_100', 'concept_replacement_100','negation_100','discourse_100','sentiment_100','casual_100', 'dialectal_100']
results_df['modification'] = pd.Categorical(results_df['modification'], categories=modification_name, ordered=True)
results_df = results_df.sort_values(by='modification')

# Calculate averages across all modifications
avg_original = results_df['original_res'].mean()
avg_modified = results_df['modified_res'].mean()
avg_difference = avg_original - avg_modified
avg_pct_difference = results_df['pct_difference'].mean()

# Add averages as a new row
results_df.loc[len(results_df)] = {
    'task': 'dialogue_contradiction_detection',
    'modification': 'average',
    'original_res': round(avg_original, 2),
    'modified_res': round(avg_modified, 2),
    'difference': -round(avg_difference, 2),
    'pct_difference': round(avg_pct_difference, 2),
    'p_value': None  # No p-value for average row
}

print("\n")
results_df.to_csv('results/dialogue/gpt4o-DP.csv')

# Apply styling to highlight rows where original_res > modified_res and significant p-values
def highlight_drops_and_significance(row):
    colors = [''] * len(row)
    if row['original_res'] > row['modified_res']:
        colors = ['background-color: red'] * len(row)
        # If p-value < 0.05, add bold text
        if 'p_value' in row and row['p_value'] is not None and row['p_value'] < 0.05:
            colors = ['background-color: red; font-weight: bold'] * len(row)
    return colors

results_df.round(2).style.apply(highlight_drops_and_significance, axis=1)


In [ ]:
# Filter for negation modification only
# negation_results = pd.read_csv('results/dialogue/gpt4o-DP.csv')
# negation_df = negation_results[negation_results['modification'] == 'negation_100']

# Load the detailed results for negation
negation_detailed = pd.read_csv('results/dialogue/gpt4o-0shot-negation_100.csv')

# Calculate accuracy for original and modified examples by type
type_results = []
for type_name in negation_detailed['type'].unique():
    type_data = negation_detailed[negation_detailed['type'] == type_name]
    
    # Calculate original and modified accuracies
    original_acc = (type_data['original_pred'] == type_data['original_label']).sum() / len(type_data)
    modified_acc = (type_data['modified_pred'] == type_data['modified_label']).sum() / len(type_data)
    
    # Calculate differences
    difference = -(original_acc - modified_acc)
    pct_difference = -round((original_acc - modified_acc) / original_acc * 100, 2)

    # Perform t-test
    original_correct = (type_data['original_pred'] == type_data['original_label']).astype(int)
    modified_correct = (type_data['modified_pred'] == type_data['modified_label']).astype(int)
    _, p_value = stats.ttest_rel(original_correct, modified_correct)
    
    type_results.append({
        'type': type_name,
        'num_samples': len(type_data),
        'original_res': round(original_acc * 100, 2),
        'modified_res': round(modified_acc * 100, 2),
        'difference': round(difference * 100, 2),
        'pct_difference': round(pct_difference, 2),
        'p_value': p_value
    })

# Create and display type-based results dataframe
type_results_df = pd.DataFrame(type_results)

# Calculate averages
avg_original = type_results_df['original_res'].mean()
avg_modified = type_results_df['modified_res'].mean()
avg_difference = avg_original - avg_modified
avg_pct_difference = type_results_df['pct_difference'].mean()
total_samples = type_results_df['num_samples'].sum()

# # Add averages row
# type_results_df.loc[len(type_results_df)] = {
#     'type': 'average',
#     'num_samples': total_samples,
#     'original_res': round(avg_original, 2),
#     'modified_res': round(avg_modified, 2),
#     'difference': round(avg_difference, 2),
#     'pct_difference': round(avg_pct_difference, 2),
#     'p_value': None
# }

# Apply the same styling as before
styled_type_results = type_results_df.round(2).style.apply(highlight_drops_and_significance, axis=1)
styled_type_results


In [ ]:
# Load results from different models
gpt4_df = pd.read_csv('results/dialogue/gpt4o-0shot-dialogue.csv')
claude_df = pd.read_csv('results/dialogue/claude-3-5-sonnet-0shot-dialogue.csv')
mixtral_df = pd.read_csv('results/dialogue/mixtral-8x22b-0shot-dialogue.csv')

# Calculate accuracy between predictions and labels
gpt4_acc = (gpt4_df['pred'] == gpt4_df['label']).mean()
claude_acc = (claude_df['pred'] == claude_df['label']).mean()
mixtral_acc = (mixtral_df['pred'] == mixtral_df['label']).mean()
# Calculate average accuracy for each model
print(f"GPT-4 Average Accuracy: {gpt4_acc:.2%}")
print(f"Claude-3.5 Average Accuracy: {claude_acc:.2%}")
print(f"Mixtral Average Accuracy: {mixtral_acc:.2%}")

# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Model': ['GPT-4', 'Claude-3.5', 'Mixtral'],
    'Accuracy': [gpt4_acc, claude_acc, mixtral_acc]
})

# Style the dataframe
def highlight_max(s):
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

styled_df = comparison_df.style.apply(highlight_max, subset=['Accuracy'])
styled_df


# Llama-3-405B

In [360]:
lm = dspy.LM('together_ai/meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo')
dspy.configure(lm=lm)